![sutd](sutd.png)
## <center>50.040 Natural Language Processing, Summer 2020<center>
<center>**Homework 4**

<center>**Due 31 July 2020, 5pm** <center>

**Write your student ID and name**

ID: 1003056

Name: Ivan Christian

Students whom you have discussed with (if any): Ng Jen Yang

### Requirements:
- Use Python to complete this homework.
- Please list students with whom you have discussed (if any).
- Follow the honor code strictly.

In this homework, we'll implement ``IBM Model 1`` using the ``expectation–maximization (EM)`` algorithm. We need to estimate the  translation probabilities  $t(f|e)$ on a parallel corpus, where $e$ is a word from the English sentences and $f$ is a word from the corresponding foreign sentences. 

Note that there's a constraint for such probabilities:
$$\sum_f t(f|e)=1 , \ \ \  t(f|e) \ge 0  \quad (1)$$

**We'll use this constraint when initializing the translation probabilities in subsequent tasks.**

## Data
We'll use the English-French parallel corpus under the folder ``data/part1``, which contains a set of translation instances. As can be seen below each instance consists of an English-French sentence pair (note that we are translating from French into English, but as we discussed in class, when working on the translation model using IBM model 1, we are interested in generating French from English).



    Hop in.	Montez.
    Hug me.	Serre-moi dans tes bras !
    I left.	Je suis parti.

The dataset is obtained from [MXNET](http://data.mxnet.io/data/fra-eng.zip). Please run the provided code below to obtain the preprocessed English sentences and French sentences. Do not perform any further preprocessing. 


In [1]:
import seaborn as sns
import numpy as np
from time import time
from collections import Counter, defaultdict

from matplotlib import pyplot as plt
%matplotlib inline

# Part 1: Statistical Machine Translation \[25 points]

In [2]:
path = 'data/part1/en-fr.txt'
with open(path, 'r', encoding='utf8') as f:
    raw_text = f.read()

#Original code from 
#https://www.d2l.ai/chapter_recurrent-neural-networks
def preprocess_nmt(text):
    '''
    Arg:
        text: parallel text, string
    Return:
        out: preprocessed text, string
    '''
    text = text.replace('\u202f', ' ').replace('\xa0', ' ')
    no_space = lambda char, prev_char: (
        True if char in (',', '!', '.') and prev_char != ' ' else False)
    out = [' '+char if i > 0 and no_space(char, text[i-1]) else char
           for i, char in enumerate(text.lower())]
    out = ''.join(out)
    return out

def tokenize_nmt(text, num_examples = None):
    '''
    Args:
        text: parallel text, string
        num_examples: number of examples to be selected, int
    Returns:
        left: English sentences, list
        right: French sentences, list
    '''
    left, right = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples: break
        parts = line.split('\t')
        if len(parts) == 2:
            left.append(parts[0].split(' '))
            right.append(parts[1].split(' '))
    return left, right


In [3]:
#English sentences and corresponding French sentences
#Each sentence has been preprocessed and tokenized
text = preprocess_nmt(raw_text)
english_sents, french_sents = tokenize_nmt(text)

In [4]:
english_sents[:10], french_sents[:10]

([['go', '.'],
  ['hi', '.'],
  ['run', '!'],
  ['run', '!'],
  ['who?'],
  ['wow', '!'],
  ['fire', '!'],
  ['help', '!'],
  ['jump', '.'],
  ['stop', '!']],
 [['va', '!'],
  ['salut', '!'],
  ['cours', '!'],
  ['courez', '!'],
  ['qui', '?'],
  ['ça', 'alors', '!'],
  ['au', 'feu', '!'],
  ['à', "l'aide", '!'],
  ['saute', '.'],
  ['ça', 'suffit', '!']])


### Quesiton 1 (3 points)
1. Implement ``word_pairs_in_corpus`` which finds out all the possible word pairs (alignments) $(e, f)$ that appear in all the instances of the English-French dataset ``english_sents``, ``french_sents``. Note that we need to pad each English sentence with the special token "NULL" at the beginning.
2. List down the 10 most frequent pairs. 
3. Count the number of unique pairs.

In [6]:
def word_pairs_in_corpus(en_sents, fr_sents):
    '''
    params:
        en_sents: list[list[str]]
        fr_sents: list[list[str]]
    return:
        align_counts: Dict()--- key: (english_word, french_word), value: counts of the word pair in the corpus
    '''
    align_counts = None
    # YOUR CODE HERE
    align_counts = defaultdict()
#     print(len(en_sents), len(fr_sents))
    
    sents_len = len(en_sents)
    for idx in range(sents_len):            
        
        for en_word in en_sents[idx]:
            for fr_word in fr_sents[idx]:
                if (en_word, fr_word) in align_counts:
                    align_counts[(en_word, fr_word)] += 1
                else:
                    align_counts[(en_word, fr_word)] = 1
    # END OF YOUR CODE
    return Counter(align_counts)

In [7]:
english_sents = [['NULL'] + sent for sent in english_sents]
align_counts = word_pairs_in_corpus(english_sents, french_sents)
align_counts.most_common(10), len(align_counts)

([(('.', '.'), 136734),
  (('NULL', '.'), 135221),
  (('i', '.'), 43189),
  (('NULL', 'je'), 39821),
  (('.', 'je'), 39096),
  (('NULL', 'de'), 35073),
  (('i', 'je'), 34415),
  (('to', '.'), 31647),
  (('.', 'de'), 30490),
  (('the', '.'), 29170)],
 1402126)

In [8]:
en_vocab = set([item[0] for item in align_counts.keys()])
fr_vocab = set([item[1] for item in align_counts.keys()])

In [9]:
len(en_vocab), len(fr_vocab)

(17430, 29741)

### Question 2 (2 points):

Implment the ``corpus_log_prob `` that computes the log probability of the corpus

In [10]:
def corpus_log_prob(en_sents, fr_sents, t):
    '''
    params:
        en_sents: list[list[str]]
        fr_sents: list[list[str]]
        t: Dict() --- contains translation probabilities. For example, t[(english_word, french_word)] = p
    return:
        logp: float --- log probability of the corpus
    '''
    logp = 0
    ### YOUR CODE HERE
    
    for en_sent, fr_sent in zip(en_sents, fr_sents):
        for en_word in en_sent:
            for fr_word in fr_sent:
                p = t[(en_word, fr_word)]
                t[(en_word, fr_word)] = np.log(p)
                
                logp += t[(en_word, fr_word)]
    
    # END OF YOUR CODE
    return logp

## Hard EM algorithm

### Question 3 (10 points)
Based on the word pairs obtained in Q1, implement ``Hard EM algorithm`` to calculate the  translation probabilities  $t(f|e)$ on the English-French corpus. 

It is possible that in the hard EM algorithm a word $\tilde{e}$ from an English sentence may not be aligned with any word from the corresponding French sentence. In this case, let us set the corresponding probabilities $t(f|\tilde{e})=\frac{1}{|V_f|}$ where $|V_f|$ is the size of the French vocabulary (in this case, the number of unique French words that ever appear in the training parallel corpus).

1. Implement ``init`` function which initializes the translation probability dictionary $t$ according to equation (1). You need to use ``numpy.random.rand()`` in this part.
2. Implement ``hard_EM`` function which runs one ``Expectation/Maximization`` iteration.
3. Run the training code

In [17]:
def init(word_pairs):
    '''
    Use np.random.rand() to initialize translation probabilities t(f|e)
    params:
        word_pairs: List[(str, str)] --- list of word pairs
    return:
        t: Dict(), key: (english_word, french_word), value: the initial probability t(f|e). For example, t[(a, un)] = 0.5
    '''
    np.random.seed(5)
    t = dict()
    ### YOUR CODE HERE
    from tqdm import tqdm
    
    for en_word, fr_word in word_pairs:
        t[(en_word, fr_word)] = np.random.rand()
#     for en_word, _ in tqdm(word_pairs):
#         list_p = []
#         en_key_pairs = [wordpair for wordpair in t.keys() if wordpair[0] == en_word]
#         align_pair_counts = [t[wordpair] for wordpair in t.keys() if wordpair[0] == en_word]
#         # Get the sum for all the en_word count in the dict
#         align_pair_sum = sum(align_pair_counts)
        
#         for p_val in align_pair_counts:
#             init_p = p_val / align_pair_sum
#             list_p.append(init_p)
#         adjusted_val = zip(en_key_pairs, list_p)
        
#         for key , val in adjusted_val:
#             t[key] = val

        
    ### END OF YOUR CODE
    return t

In [18]:
hard_t = init(list(align_counts.keys()))
hard_t

{('NULL', 'va'): 0.22199317108973948,
 ('NULL', '!'): 0.8707323061773764,
 ('go', 'va'): 0.20671915533942642,
 ('go', '!'): 0.9186109079379216,
 ('.', 'va'): 0.48841118879482914,
 ('.', '!'): 0.6117438629026457,
 ('NULL', 'salut'): 0.7659078564803156,
 ('hi', 'salut'): 0.5184179878729432,
 ('hi', '!'): 0.29680050157622195,
 ('.', 'salut'): 0.18772122866125163,
 ('NULL', 'cours'): 0.08074126876487486,
 ('run', 'cours'): 0.73844029619897,
 ('run', '!'): 0.4413092228959531,
 ('!', 'cours'): 0.1583098677126512,
 ('!', '!'): 0.8799370312012789,
 ('NULL', 'courez'): 0.27408646199222464,
 ('run', 'courez'): 0.4142350190810513,
 ('!', 'courez'): 0.29607993273364797,
 ('NULL', 'qui'): 0.6287879088794833,
 ('NULL', '?'): 0.579837810189545,
 ('who?', 'qui'): 0.5999291966249876,
 ('who?', '?'): 0.26581911753550724,
 ('NULL', 'ça'): 0.2846858806413638,
 ('NULL', 'alors'): 0.2535882057737875,
 ('wow', 'ça'): 0.3275639476887341,
 ('wow', 'alors'): 0.14416430065342045,
 ('wow', '!'): 0.165612861201267

In [24]:
def hard_EM(en_sents, fr_sents, fr_vocab, t):
    '''
    One 'Expectation', 'Maximization' iteration.
    params:
        en_sents: List[List[str]]
        fr_sents: List[List[str]]
        fr_vocab: int --- size of the French vocab
        t: Dict() --- translation probability dictionary from last iteration
        
    return:
        new_t: Dict() --- updated parameters, dictionary
    '''
    new_t = t
    ### YOUR CODE HERE
    
    for en_sent, fr_sent in zip(en_sents, fr_sents):
        print(en_sent, fr_sent)
        break
    
    
    
    ### END OF YOUR CODE
    
    return new_t

In [25]:
hard_t = hard_EM(english_sents, french_sents, fr_vocab, hard_t)  

['NULL', 'go', '.'] ['va', '!']


In [20]:
############################################################
# Randomly initialized the probabilities under the contraint
############################################################
hard_t = init(list(align_counts.keys()))
    
##################
# Hard EM training
##################
iteration = 0
while iteration < 10:
    
    logp = corpus_log_prob(english_sents, french_sents, hard_t)    
    hard_t = hard_EM(english_sents, french_sents, fr_vocab, hard_t)  
    print('Objective Function:', round(logp, 5))
    iteration += 1

C:\Users\chris\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in log
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

### Visualization
Using 2D-heatmap, visualize the translation probability (namely $t(f|e)$) for each of the instances below:

    NULL tom loves chocolate .   tom adore le chocolat .    
    NULL it was a very exciting game .   c'était un jeu vraiment très excitant .  

In [ ]:
def visualize_trans_prob(en, fr, t):
    '''
    Visualize the translation probability of an instance
    '''
    alignments = np.zeros([len(fr), len(en)])
    for i in range(len(fr)):
        for j in range(len(en)):
            alignments[i, j] = t[(en[j], fr[i])]
    sns.heatmap(alignments, cmap='PuBuGn', annot=True)
    _, _ = plt.yticks(np.arange(len(fr))+0.5, fr, rotation=0, fontsize=10)
    _, _ = plt.xticks(np.arange(len(en))+0.5, en, rotation=30, fontsize=10)

In [ ]:
en = "NULL tom loves chocolate .".split()
fr = "tom adore le chocolat .".split()
visualize_trans_prob(en, fr, hard_t)

In [ ]:
plt.figure(figsize=(8, 6))
en = "NULL it was a very exciting game .".split()
fr = "c'était un jeu vraiment très excitant .".split()
visualize_trans_prob(en, fr, hard_t)

## Soft EM algorithm

### Question 4 (10 points)

1. Implement ``soft_EM`` function which runs one ``Expectation/Maximization`` iteration.
2. Run the training code

In [ ]:
def soft_EM(en_sents, fr_sents, t):
    '''
    params:
        en_sents: English sentences, list
        fr_sents: foreign sentences, list
    return:
        t: updated parameters, dictionary
    '''
    new_t = t
    ### YOUR CODE HERE
    
    ### END OF YOUR CODE
    return new_t

Let us check the algorithm first using the objective value.

In [ ]:
############################################################
# Randomly initialized the probabilities under the contraint
############################################################
soft_t = init(list(align_counts.keys()))
    
##################
# Hard EM training
##################
iteration = 0
while iteration < 15:
    
    logp = corpus_log_prob(english_sents, french_sents, soft_t)    
    soft_t = soft_EM(english_sents, french_sents, soft_t)  
    print('Objective Function:', round(logp, 5))
    iteration += 1

### Visualization

In [ ]:
en = "NULL tom loves chocolate .".split()
fr = "tom adore le chocolat .".split()
visualize_trans_prob(en, fr, soft_t)

In [ ]:
plt.figure(figsize=(8,6))
en = "NULL it was a very exciting game .".split()
fr = "c'était un jeu vraiment très excitant .".split()
visualize_trans_prob(en, fr, soft_t)